In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
from matplotlib import pyplot as plt
from math import sqrt
from scipy.spatial import distance
import seaborn as sns

import haversine as hs

import os
import pickle

import warnings
warnings.filterwarnings('ignore')

import cartoframes
import json

import folium

from bs4 import BeautifulSoup
import requests

from shapely import wkt

from pandas.io.json import json_normalize

import re
import io

import time
from tqdm.notebook import tqdm
from tqdm import tqdm_gui
import sys

from scipy.spatial import distance

from sklearn import preprocessing

import geopandas as gpd

import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

import time
from tqdm.notebook import tqdm
from tqdm import tqdm_gui
import sys

from bs4 import BeautifulSoup
import requests

from functools import partial
from operator import is_not

import statistics as st

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%matplotlib inline

In [ ]:
# Подготовка переменных для работы с API Яндекса
url = 'https://search-maps.yandex.ru/v1/\
?text={}\
&type=biz\
&lang=ru_RU\
&ll={},{}\
&spn={},{}\
&rspn=1\
&results=500\
&skip={}\
&apikey=e6a410e9-68b3-406f-a59f-dad0a65810ef'



directory_dict = {
    'dentistry': 'Стоматологическая клиника',
    'dental_clinic': 'Стоматологическая поликлиника',
    'metro': 'Станция метро',
    'mall': 'Торговый центр',
    'entertainment_center': 'развлекательный центр',
    'rail_station': 'Железнодорожная станция',
    'business_center': 'Бизнес-центр'
    }

# 4 крайние координаты в нашем диапазоне
lst = ((55.753372, 37.919860),(55.775826, 37.301879),(55.935639, 37.588634),(55.547931, 37.631398))

# центральная
center=[55.751439, 37.624187]

prohibitedWords = ['properties.', 'CompanyMetaData.']

# lst_of_df = [transport_bus, dentistry, dental_clinic, adult_clinic, children_clinic,
#                      mfc_mydocuments, parking, pharmacy, medical_laboratory,
#                      school, private_school, kindergarten, post_office, product_shop, metro,
#                      shopping_center, business_center, hospital, bank]

In [ ]:
# на вход tuple tupl-ов ((a,b), (c,d))
# получаем центральную координату
def centroid(vertexes):
    _x_list = [vertex [0] for vertex in vertexes]
    _y_list = [vertex [1] for vertex in vertexes]
    _len = len(vertexes)
    _x = sum(_x_list) / _len
    _y = sum(_y_list) / _len
    return(_x, _y)

# расчет сдвига для API Яндекса
# на вход tuple tupl-ов из крайних координат пространства + координаты центральной точни
def min_max_spn(lst_values, lat_c, lng_c):
    max_delta_lat = 0.0
    max_delta_lng = 0.0
    for value in lst:
        delta_lat = abs(lat_c - value[0])
        delta_lng = abs(lng_c - value[1])
        if delta_lat > max_delta_lat:
            max_delta_lat = delta_lat
        if delta_lng > max_delta_lng:
            max_delta_lng = delta_lng
    return (max_delta_lat, max_delta_lng)

# функция поиска новых координат при помощи сдвига долготы и широты
def points_by_shift(lattitude1, longitude1, delta_lat, delta_long):
    return [(lattitude1 + delta_lat, longitude1 + delta_long),
            (lattitude1 - delta_lat, longitude1 - delta_long),
            (lattitude1 + delta_lat, longitude1 - delta_long),
            (lattitude1 - delta_lat, longitude1 + delta_long)
           ]

# создаю таблицы с названием из словаря выше
def table_creation(url, directory_dict, lst, spn = [], centr = [], skip=0):
    if not centr:
        centr = centroid(lst)
    if not spn:
        spn = min_max_spn(lst, centr[0], centr[1])
#     i = 0
    for layer in directory_dict.keys():
        
        url_new = url.format(directory_dict[layer],
                            round(centr[1], 8),
                            round(centr[0], 8),
                            round(spn[1], 8),
                            round(spn[0], 8),
                            skip
                            )

        response = requests.get(url_new)
        data = response.json()

        try:
            globals()[layer] = json_normalize(data["features"])
        except:
            pass

#         i+=1
#         print(f'[{i}/{len(directory_dict)}] LOADED {layer}')

# функция перевода списка координат в тип геометрия
def add_points(dadaframe, geo_column_name='geometry.coordinates'):
    
    dadaframe['lat'] = dadaframe[geo_column_name].apply(lambda x: x[1])
    dadaframe['lng'] = dadaframe[geo_column_name].apply(lambda x: x[0])
    
    return gpd.GeoDataFrame(dadaframe, geometry=gpd.points_from_xy(dadaframe.lng, dadaframe.lat))

# создаю таблицу со всеми датафреймами
def creation_yandex_table(directory_dict, lst_of_df):
    lst_of_keys = []
    
    for i in directory_dict.keys():
        lst_of_keys.append(i)
    
    # присоединяю столбец с типом поиска
    for j in range(0, len(lst_of_df)):
        lst_of_df[j]['search_result'] = directory_dict[lst_of_keys[j]]

    result_table = lst_of_df[0].append(lst_of_df[1:], ignore_index=True)

    big_regex = re.compile('|'.join(map(re.escape, prohibitedWords)))
    result_table = result_table.rename(columns={i: big_regex.sub("", i) for i in result_table.columns})
    return add_points(result_table)

# проверка границ локации для выгрузки по API с яндекса
def create_map(lst, center_of_map, zoom_start=12):
    m = folium.Map(location=center_of_map, zoom_start=12, tiles='OpenStreetMap')
    
    lst = gpd.GeoSeries(lst)
    
    folium.Marker(
      location=center_of_map
     ).add_to(m)
    
    for row in lst:
        
        folium.CircleMarker(
            location= [row[0],row[1]],
            radius=5,
            color='green',
            fill=True,
            fill_color='green'
            ).add_to(m)
        
    return m

# рисует карту с координатами объектов, которые выгрузили при помощи запроса creation_yandex_table
# dent- это таблица, которую нужно отрисовывать
def create_map_v2(dent):
    m = folium.Map(location=[dent.lat.mean(), dent.lng.mean()], zoom_start=12, tiles='OpenStreetMap')
    
    for _, row in dent.iterrows():
        
        folium.CircleMarker(
            location= [row['lat'],row['lng']],
            radius=5,
            popup= row[['name','Hours.text']],
            color='green',
            fill=True,
            fill_color='green'
            ).add_to(m)
        
    return m

# создаю таблицу в google sheet и заполняю её данными
#table_name- название таблицы для сохранения
#spreadsheet_key- беру из url таблицы
#wks_name- название work_sheet
#df_clustering- таблица для сохранения, с итоговой кластеризацией
#credentials- это инициализируется при авторизации в GS
#mail_to_work- почта, кому открыть доступ
def gs_table_creating(table_name, mail_to_work):
    #table_name = 'clustering_data'
    sheet = gs.create(table_name)
    
    #mail_to_work = 'data.gnathos@gmail.com'
    sheet.share(mail_to_work, perm_type='user', role='writer')
    
def gs_table_uploading(spreadsheet_key, wks_name, df_clustering, credentials):
    d2g.upload(df_clustering, spreadsheet_key, wks_name, credentials=credentials, row_names=True)



def save_xlsx(df, file_name, sheet_name='Sheet_name_1'):
    df.to_excel(file_name, sheet_name)  

In [ ]:
spn = (0.3, 0.5)
shift_points = points_by_shift(center[0], center[1], spn[0], spn[1])
m= create_map(shift_points, center)
m

In [ ]:
itter = 0
frames = []

while itter <= 10:
    
    skip = 500 * itter
    table_creation(url, directory_dict, lst, spn, center, skip)
    lst_of_df = [dentistry, dental_clinic, metro, mall, entertainment_center, rail_station, business_center]
    
    temp_table = creation_yandex_table(directory_dict, lst_of_df)
    frames.append(temp_table)
    
    if temp_table['search_result'].value_counts().max() != 500:
        break
        
    itter+=1
    
ya_table = pd.concat(frames)

In [ ]:
if ya_table.columns.values[3] == 'name':
    ya_table.columns.values[3] = 'name_v1'
if ya_table.columns.values[7] == 'name':
    ya_table.columns.values[7] = 'name_v2'

In [ ]:
ya_table.loc[ya_table['search_result'] == 'Стоматологическая клиника'].head()

In [ ]:
def create_map_v2(dent):
    m = folium.Map(location=[dent.lat.mean(), dent.lng.mean()], zoom_start=12, tiles='OpenStreetMap')
    
    for _, row in dent.iterrows():
        
        folium.CircleMarker(
            location= [row['lat'],row['lng']],
            radius=5,
            popup= row[['name_v1','Hours.text']],
            color='green',
            fill=True,
            fill_color='green'
            ).add_to(m)
        
    return m

In [ ]:
mm= create_map_v2(ya_table.loc[ya_table['search_result'] == 'Станция метро'])
mm